In [1]:
#This notebook will contain calculations and analysis for the travel time of an average subway rider vs. an handicapped rider

In [2]:
#First I have two lists of random NYC addresses
import pandas as pd
import numpy as np

A_addresses = pd.read_csv('NYC_addresses_1.csv')
B_addresses = pd.read_csv('NYC_addresses_2.csv')

In [4]:
A_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce
0,"133-20 85 ST NEW YORK, NY 11417",133-20,85 ST,NEW YORK,NY,11417,-73.851338,40.673495,36,81,5800,3
1,"292 MONTAUK AVE NEW YORK, NY 11208",292,MONTAUK AVE,NEW YORK,NY,11208,-73.877478,40.671479,36,47,119400,3
2,"6 ROMER RD NEW YORK, NY 10304",6,ROMER RD,NEW YORK,NY,10304,-74.103098,40.593125,36,85,17701,3
3,"164-10 104 ST NEW YORK, NY 11414",164-10,104 ST,NEW YORK,NY,11414,-73.827662,40.651995,36,81,88400,6
4,"1429 SHORE PKWY NEW YORK, NY 11214",1429,SHORE PKWY,NEW YORK,NY,11214,-74.005699,40.598670,36,47,28200,1


In [5]:
B_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce
0,"3236 BOSTON RD NEW YORK, NY 10469",3236,BOSTON RD,NEW YORK,NY,10469,-73.854792,40.872162,36,5,34800,1
1,"970 AMSTERDAM AVE NEW YORK, NY 10025",970,AMSTERDAM AVE,NEW YORK,NY,10025,-73.965107,40.801581,36,61,19500,4
2,"1623 DECATUR ST NEW YORK, NY 11385",1623,DECATUR ST,NEW YORK,NY,11385,-73.901360,40.694898,36,81,55900,1
3,"12-06 151 PL NEW YORK, NY 11357",12-06,151 PL,NEW YORK,NY,11357,-73.811475,40.791270,36,81,98700,2
4,"135 BCH 56 PL NEW YORK, NY 11692",135,BCH 56 PL,NEW YORK,NY,11692,-73.787031,40.590884,36,81,97202,1


In [73]:
origins = A_addresses['address_full'].tolist()
origins

['133-20 85 ST NEW YORK, NY 11417 ',
 '292 MONTAUK AVE NEW YORK, NY 11208 ',
 '6 ROMER RD NEW YORK, NY 10304 ',
 '164-10 104 ST NEW YORK, NY 11414 ',
 '1429 SHORE PKWY NEW YORK, NY 11214 ',
 '196 22 ST NEW YORK, NY 11232 ',
 '715 SAGAMORE ST NEW YORK, NY 10462 ',
 '4730 RICHARDSON AVE NEW YORK, NY 10470 ',
 '94-17 84 ST NEW YORK, NY 11416 ',
 '2732 MILL AVE NEW YORK, NY 11234 ']

In [74]:
destinations = B_addresses['address_full'].tolist()
destinations

['3236 BOSTON RD NEW YORK, NY 10469 ',
 '970 AMSTERDAM AVE NEW YORK, NY 10025 ',
 '1623 DECATUR ST NEW YORK, NY 11385 ',
 '12-06 151 PL NEW YORK, NY 11357 ',
 '135 BCH 56 PL NEW YORK, NY 11692 ',
 '196 ROSEDALE AVE NEW YORK, NY 10312 ',
 '519 MADISON AVE NEW YORK, NY 10022 ',
 '197-26 CARPENTER AVE NEW YORK, NY 11423 ',
 '63-62 77 ST NEW YORK, NY 11379 ',
 '120-36 143 ST NEW YORK, NY 11436 ']

In [11]:
#That was the easy part. Now, gotta build an automated browser to run imput these addresses. 
import os
import random
import time

from playwright.async_api import async_playwright, expect, Keyboard

In [12]:
os.makedirs('data/', exist_ok=True)

In [13]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'

In [66]:
async def open_browser(headless=False, user_agent=user_agent):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.chromium.launch(headless=False)

    # set a user agent
    context = await browser.new_context(user_agent=user_agent)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [67]:
driver, page = await open_browser()

In [68]:
url = 'https://www.mta.info/'
await page.goto(url)

<Response url='https://www.mta.info/' request=<Request url='https://www.mta.info/' method='GET'>>

In [69]:
from_search = page.locator('//*[@id="edit-origin"]')
from_search

<Locator frame=<Frame name= url='https://www.mta.info/'> selector='//*[@id="edit-origin"]'>

In [70]:
await from_search.fill('853 Riverside Dr, New York, NY, USA')

In [71]:
await page.mouse.click(200,450)

In [65]:
to_search = page.locator('//*[@id="edit-dest"]')
to_search

<Locator frame=<Frame name= url='https://www.mta.info/'> selector='//*[@id="edit-dest"]'>

In [55]:
await to_search.fill('75 9th Ave, New York, NY, USA')

In [56]:
await page.mouse.click(0,5)

In [57]:
submit = page.locator('//*[@id="edit-submit"]')
submit

<Locator frame=<Frame name= url='https://www.mta.info/'> selector='//*[@id="edit-submit"]'>

In [58]:
await submit.click()

In [94]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('api_key')

In [109]:
import requests

curl -X POST -d '{
  "origin":{
    "address": "853 Riverside Drive, New York, NY"
      },
  "destination":{
    "address": "75 9th Avenue, New York, NY" 
      },
  "travelMode": "TRANSIT",
  "computeAlternativeRoutes": true,
  "transitPreferences": {
    "allowedTravelModes": ["SUBWAY","BUS"]
  },
  "languageCode": "en-US",
  "units": "METRIC"
 }' \
-H 'Content-Type: application/json' 
-H 'X-Goog-Api-Key: api_key' \
-H 'X-Goog-FieldMask: routes.duration,routes.distanceMeters,routes.transitDetails' \
'https://routes.googleapis.com/directions/v2:computeRoutes'

SyntaxError: unterminated string literal (detected at line 3) (1632780211.py, line 3)

In [183]:
import requests

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': api_key,
    'X-Goog-FieldMask': 'routes.legs.steps.transitDetails,routes.localizedValues'}

json_data = {
    'origin': {
        'address': '853 Riverside Drive, New York, NY',
    },
    'destination': {
        'address': '75 9th Avenue, New York, NY',
    },
    'travelMode': 'TRANSIT',
    'computeAlternativeRoutes': True,
    'transitPreferences': {
        'allowedTravelModes': [
            'SUBWAY',
            'BUS',
        ],
    },
    'languageCode': 'en-US',
    'units': 'METRIC',
}

response = requests.post('https://routes.googleapis.com/directions/v2:computeRoutes', headers=headers, json=json_data)

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
#data = '{\n  "origin":{\n    "address": "853 Riverside Drive, New York, NY"\n      },\n  "destination":{\n    "address": "75 9th Avenue, New York, NY" \n      },\n  "travelMode": "TRANSIT",\n  "computeAlternativeRoutes": true,\n  "transitPreferences": {\n    "allowedTravelModes": ["SUBWAY","BUS"]\n  },\n  "languageCode": "en-US",\n  "units": "METRIC"\n }'
#response = requests.post('https://routes.googleapis.com/directions/v2:computeRoutes', headers=headers, data=data)

In [184]:
data =response.json()

In [185]:
data

{'routes': [{'legs': [{'steps': [{},
      {},
      {},
      {},
      {'transitDetails': {'stopDetails': {'arrivalStop': {'name': '59 St-Columbus Circle',
          'location': {'latLng': {'latitude': 40.768246,
            'longitude': -73.981926}}},
         'arrivalTime': '2025-07-29T19:57:55Z',
         'departureStop': {'name': '157 St',
          'location': {'latLng': {'latitude': 40.834042,
            'longitude': -73.944893}}},
         'departureTime': '2025-07-29T19:38:55Z'},
        'localizedValues': {'arrivalTime': {'time': {'text': '3:57\u202fPM'},
          'timeZone': 'America/New_York'},
         'departureTime': {'time': {'text': '3:38\u202fPM'},
          'timeZone': 'America/New_York'}},
        'headsign': 'South Ferry',
        'headway': '300s',
        'transitLine': {'agencies': [{'name': 'MTA New York City Transit',
           'phoneNumber': '+1 718-330-1234',
           'uri': 'http://www.mta.info/'}],
         'name': '1 Train (Broadway-7 Av Local)',
  

In [214]:
import requests

headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Origin': 'https://www.mta.info',
    'Referer': 'https://www.mta.info/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

params = {
    'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
    'allowUnknownTransfers': 'false',
    'mode': 'TRANSIT,WALK',
    'numItineraries': '3',
    'showIntermediateStops': 'true',
    'smartKissAndRide': 'true',
    'flagStopBufferSize': '50',
    'nextDepartureWindow': '86400',
    'toPlace': '40.7424509, -74.0059581',
    'fromPlace': '40.8366459, -73.9471301',
    'arriveBy': 'false',
    'wheelchair': 'false',
    'maxWalkDistance': '804',
    'date': '7/29/25',
    'time': '03:55 PM',
    'optimize': 'QUICK',
    'bannedAgencies': '',
    'bannedRouteTypes': '',
}

response = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)

In [215]:
data = response.json()

In [220]:
data['plan']['itineraries'][1]['duration']

2658

In [207]:
A_addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address_full  10 non-null     object 
 1   number        10 non-null     object 
 2   street        10 non-null     object 
 3   city          10 non-null     object 
 4   state         10 non-null     object 
 5   zip           10 non-null     int64  
 6   longitude     10 non-null     float64
 7   latitude      10 non-null     float64
 8   statefp       10 non-null     int64  
 9   countyfp      10 non-null     int64  
 10  tractce       10 non-null     int64  
 11  blkgrpce      10 non-null     int64  
 12  coordinates   10 non-null     object 
dtypes: float64(2), int64(5), object(6)
memory usage: 1.1+ KB


In [239]:
A_addresses['coordinates'] = A_addresses["latitude"].astype(str) + ", " + A_addresses["longitude"].astype(str)
A_addresses

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce,coordinates
0,"133-20 85 ST NEW YORK, NY 11417",133-20,85 ST,NEW YORK,NY,11417,-73.851338,40.673495,36,81,5800,3,"40.673495, -73.851338"
1,"292 MONTAUK AVE NEW YORK, NY 11208",292,MONTAUK AVE,NEW YORK,NY,11208,-73.877478,40.671479,36,47,119400,3,"40.671479, -73.877478"
2,"6 ROMER RD NEW YORK, NY 10304",6,ROMER RD,NEW YORK,NY,10304,-74.103098,40.593125,36,85,17701,3,"40.593125, -74.103098"
3,"164-10 104 ST NEW YORK, NY 11414",164-10,104 ST,NEW YORK,NY,11414,-73.827662,40.651995,36,81,88400,6,"40.651995, -73.827662"
4,"1429 SHORE PKWY NEW YORK, NY 11214",1429,SHORE PKWY,NEW YORK,NY,11214,-74.005699,40.598670,36,47,28200,1,"40.59867, -74.005699"
5,"196 22 ST NEW YORK, NY 11232",196,22 ST,NEW YORK,NY,11232,-73.995537,40.661591,36,47,14500,2,"40.661591, -73.995537"
6,"715 SAGAMORE ST NEW YORK, NY 10462",715,SAGAMORE ST,NEW YORK,NY,10462,-73.866310,40.848674,36,5,23000,1,"40.848674, -73.86631"
7,"4730 RICHARDSON AVE NEW YORK, NY 10470",4730,RICHARDSON AVE,NEW YORK,NY,10470,-73.850599,40.904675,36,5,41400,2,"40.904675, -73.850599"
8,"94-17 84 ST NEW YORK, NY 11416",94-17,84 ST,NEW YORK,NY,11416,-73.856868,40.685023,36,81,3400,1,"40.685023, -73.856868"
9,"2732 MILL AVE NEW YORK, NY 11234",2732,MILL AVE,NEW YORK,NY,11234,-73.914308,40.607894,36,47,70201,5,"40.607894, -73.914308"


In [240]:
B_addresses['coordinates'] = B_addresses["latitude"].astype(str) + ", " + B_addresses["longitude"].astype(str)
B_addresses

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce,coordinates
0,"3236 BOSTON RD NEW YORK, NY 10469",3236,BOSTON RD,NEW YORK,NY,10469,-73.854792,40.872162,36,5,34800,1,"40.872162, -73.854792"
1,"970 AMSTERDAM AVE NEW YORK, NY 10025",970,AMSTERDAM AVE,NEW YORK,NY,10025,-73.965107,40.801581,36,61,19500,4,"40.801581, -73.965107"
2,"1623 DECATUR ST NEW YORK, NY 11385",1623,DECATUR ST,NEW YORK,NY,11385,-73.901360,40.694898,36,81,55900,1,"40.694898, -73.90136"
3,"12-06 151 PL NEW YORK, NY 11357",12-06,151 PL,NEW YORK,NY,11357,-73.811475,40.791270,36,81,98700,2,"40.79127, -73.811475"
4,"135 BCH 56 PL NEW YORK, NY 11692",135,BCH 56 PL,NEW YORK,NY,11692,-73.787031,40.590884,36,81,97202,1,"40.590884, -73.787031"
5,"196 ROSEDALE AVE NEW YORK, NY 10312",196,ROSEDALE AVE,NEW YORK,NY,10312,-74.195661,40.547968,36,85,17014,1,"40.547968, -74.195661"
6,"519 MADISON AVE NEW YORK, NY 10022",519,MADISON AVE,NEW YORK,NY,10022,-73.973895,40.759931,36,61,10200,1,"40.759931, -73.973895"
7,"197-26 CARPENTER AVE NEW YORK, NY 11423",197-26,CARPENTER AVE,NEW YORK,NY,11423,-73.762244,40.711682,36,81,50000,1,"40.711682, -73.762244"
8,"63-62 77 ST NEW YORK, NY 11379",63-62,77 ST,NEW YORK,NY,11379,-73.878306,40.718089,36,81,66100,1,"40.718089, -73.878306"
9,"120-36 143 ST NEW YORK, NY 11436",120-36,143 ST,NEW YORK,NY,11436,-73.795799,40.675556,36,81,18200,1,"40.675556, -73.795799"


In [241]:
A_coors = A_addresses['coordinates'].tolist()
A_coors

['40.673495, -73.851338',
 '40.671479, -73.877478',
 '40.593125, -74.103098',
 '40.651995, -73.827662',
 '40.59867, -74.005699',
 '40.661591, -73.995537',
 '40.848674, -73.86631',
 '40.904675, -73.850599',
 '40.685023, -73.856868',
 '40.607894, -73.914308']

In [242]:
B_coors = B_addresses['coordinates'].tolist()
B_coors

['40.872162, -73.854792',
 '40.801581, -73.965107',
 '40.694898, -73.90136',
 '40.79127, -73.811475',
 '40.590884, -73.787031',
 '40.547968, -74.195661',
 '40.759931, -73.973895',
 '40.711682, -73.762244',
 '40.718089, -73.878306',
 '40.675556, -73.795799']

In [ ]:
def find_regular_route(url, headers =None):
    """
    Fill in origin and destination fields and make an api call for the travel routes.
    Record the duration of each suggested route

    """
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        data = response.json()

        print(data['plan)
        print(f"API response status: {response.status_code}")

        df = pd.json_normalize(data)
        df.to_csv(f'/Users/zoeslemmons/Desktop/MTA-elevators/outage_data/outages_{timestamp}.csv', index = False)

        return timestamp, data

        # You can also store the timestamp and other data (e.g., response content)
        # in a log file, database, or a data structure like a list of dictionaries.
        # Example:
        # log_entry = {"timestamp": timestamp.isoformat(), "url": url, "status_code": response.status_code}
        # with open("api_logs.txt", "a") as f:
        #     f.write(str(log_entry) + "\n")

    except requests.exceptions.RequestException as e:
        print(f"API call to {url} failed at: {timestamp} - Error: {e}")

In [253]:
durations=[]
for A_loc, B_loc in zip(A_coors, B_coors):
    params = {
        'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
        'allowUnknownTransfers': 'false',
        'mode': 'TRANSIT,WALK',
        'numItineraries': '3',
        'showIntermediateStops': 'true',
        'smartKissAndRide': 'true',
        'flagStopBufferSize': '50',
        'nextDepartureWindow': '86400',
        'toPlace': B_loc,
        'fromPlace': A_loc,
        'arriveBy': 'false',
        'wheelchair': 'false',
        'maxWalkDistance': '804',
        'date': '7/29/25',
        'time': '03:55 PM',
        'optimize': 'QUICK',
        'bannedAgencies': '',
        'bannedRouteTypes': '',
            }
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Origin': 'https://www.mta.info',
        'Referer': 'https://www.mta.info/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        }
    response = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)
    data = response.json()
    print(data['plan']['itineraries'][0]['duration'])
    durations.append(data['plan']['itineraries'][0]['duration'])
        

6564
4537
8447
6303
6736
6321
3078
6839
2178
7519


In [256]:
durations

[6564, 4537, 8447, 6303, 6736, 6321, 3078, 6839, 2178, 7519]

In [270]:
durations_accessible=[]
for A_loc, B_loc in zip(A_coors, B_coors):
    params = {
        'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
        'allowUnknownTransfers': 'false',
        'mode': 'TRANSIT,WALK',
        'numItineraries': '3',
        'showIntermediateStops': 'true',
        'smartKissAndRide': 'true',
        'flagStopBufferSize': '50',
        'nextDepartureWindow': '86400',
        'toPlace': B_loc,
        'fromPlace': A_loc,
        'arriveBy': 'false',
        'wheelchair': 'true',
        'maxWalkDistance': '804',
        'date': '7/29/25',
        'time': '03:55 PM',
        'optimize': 'QUICK',
        'bannedAgencies': '',
        'bannedRouteTypes': '',
            }
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Origin': 'https://www.mta.info',
        'Referer': 'https://www.mta.info/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        }
    response_accessible = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)
    data_accessible = response_accessible.json()
    print(data_accessible['plan']['itineraries'][0]['duration'])
    durations_accessible.append(data_accessible['plan']['itineraries'][0]['duration'])

8831
6305
8447
6303
6736
8101
3419
9356
2178
7519


In [271]:
durations_accessible

[8831, 6305, 8447, 6303, 6736, 8101, 3419, 9356, 2178, 7519]

In [272]:
import pandas as pd

df = pd.DataFrame({'commute_time' : durations})
df['accessible_commute_time'] = pd.DataFrame(durations_accessible)
df

,commute_time,accessible_commute_time
0,6564,8831
1,4537,6305
2,8447,8447
3,6303,6303
4,6736,6736
5,6321,8101
6,3078,3419
7,6839,9356
8,2178,2178
9,7519,7519


In [273]:
df['time_difference'] = df['accessible_commute_time'] - df['commute_time']
df

,commute_time,accessible_commute_time,time_difference
0,6564,8831,2267
1,4537,6305,1768
2,8447,8447,0
3,6303,6303,0
4,6736,6736,0
5,6321,8101,1780
6,3078,3419,341
7,6839,9356,2517
8,2178,2178,0
9,7519,7519,0
